# ChatData

## Sqlite
We are using a file system called **sqlite**. It looks and acts like a real single user relational database (RDB). sqlite3 comes packaged with python. You do not need to install the library. You simply import it (as below).

- We are using Pandas to write and read to sqlite. Pandas will manage a lot of the complexity of dealing with a RDB. There are other ways of reading and writing to a RDB that are VERY common and often used in production systems.  

- The most common other way is to read or write to the RDB row by row. As you can see Pandas puts the entire dataframe into the RDB or extracts a new dataframe from the RDB. These are not row by row operations. These are set operations. Set operations ARE more efficient. However, it is common to use row by row operations to avoid needing large memory computers to hold the dataframes. One row only takes a few bytes. An entire dataframe could be many gigabytes and even petabytes of data. Obviously you will end up with problems with your compute resource if your files are this big. This will not happen to you in this activity and may never happen to you while you are a data analyst. Please just be aware of this. 

- This activity is primarily about querying a RDB, so, we are using a simple way that Pandas provides. If you want to learn the more complex way, just google 'reading and wrting to a relational database using python'. There are many resources to learn from.

- The data types that sqlite supports are quite limited. for example, it does not have a DATE type.  This is not a challenge for this project. However, more sophisticated database systems,such as Postgresql, have a large array of data types, such as "DATE", etc. that give those systems additional capability. 

- All of the SQL queries could also be performed on the Pandas DataFrames directly.  You may want to try this yourself for comparison (but make sure you do the SQL queries first, as this is an exercise in using SQL!).


## SQL Magic
Within the Jupyter notebook we will be using something called **SQL Magic**.  This provides a convenient way to write SQL queries directly into code cells in the notebook and to read the results back into a Pandas DataFrame.  This makes working with SQL much easier!

Note that you may need to update your sqlite version using `conda update -c anaconda sqlite` in order for the SQL Magic to work correctly.

## The Data Analysis Lifecycle
The sections in this notebook follow the stages of the Data Analysis Lifecycle introduced in an earlier activity.  The stages are:

- Acquire
- Transform
- Organise
- Analyse
- Communicate
- Maintain


The requirements document for this project is Template SQL queries.xlsx.

# Preliminary Steps: Create a Database
First let's import Sqlite and the other libraries we will need.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3

### Create the Database
Now we will create the Sqlite database.  Here is some code that does this for you.  We use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

In [2]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql 

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db 

#### Drop the `queries` table if it already exists
The queries table will be our record of the queries created to answer the questions from the requirements spreadsheet.  As we will be running this Jupyter notebook a few times, let's drop (i.e. remove) the queries table so that we start fresh each time.  Here is the code to do this.  We use the `%%sql` magic command to tell Jupyter that we are going to write SQL in the cell.


In [3]:
%%sql
DROP TABLE IF EXISTS 'queries'

 * sqlite:///chatdata.db
Done.


[]

# Task 1: Load the Data
Now we can start loading the data.  The tables will be created as we load the data.  

Note that some of these files are quite large, so make sure you have plenty of free memory!

## Lifecycle Stages: Transform and Organise
The data has already been processed into 3 clean data files ready for this project:

- users.csv
- posts.csv
- comments.csv

We will load these files into our database.

## Lifecycle Stage: Acquire
The data can be found in the OpenClassrooms instructions for this activity.

### Load Comments Data into a `comments` table
Now we will load the data from the csv files into our sqlite database.

First we load the csv file into a Pandas dataframe:

In [4]:
comments = pd.read_csv('comments.csv')
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


Now take the comments dataframe and push the data into the Sqlite database table called 'comments':

In [5]:
# load comments into sqlite
comments.to_sql('comments', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM comments'
comments = pd.read_sql(sql, con)
comments

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141
...,...,...,...,...,...,...
49995,791747,424214,1,Agreed. I probably should have done more than ...,2019-08-29 18:59:33,85665
49996,791748,424201,0,Of course cost parameter exists on other ml al...,2019-08-29 19:05:37,35291
49997,791749,203687,0,What's maybe not quite clear from that linked ...,2019-08-29 19:06:35,27340
49998,791752,137072,0,GLMs are fit using iteratively reweighted leas...,2019-08-29 19:25:35,27340


### Load Other CSVs
Now that you have seen the code for loading in the comments.csv now do the same to read and write the posts.csv and users.csv to sqlite. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. Insert as many cells as you want to do this.
    </font></div>

In [6]:
# load the CSV file
posts = pd.read_csv('posts.csv')
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,NaN,...,NaN,NaN,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,NaN
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,NaN,...,NaN,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,NaN,NaN
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,NaN,...,NaN,NaN,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,NaN,NaN
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,NaN,...,NaN,NaN,2019-08-24 09:57:01,NaN,NaN,0,0,0,NaN,NaN
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,NaN,...,NaN,2019-08-24 12:13:46,2019-08-24 12:13:46,NaN,NaN,0,1,0,NaN,NaN


In [7]:
# load posts into sqlite
posts.to_sql('posts', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM posts'
posts = pd.read_sql(sql, con)
posts

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42229,438739,2,0,270897,2019-12-01 03:52:33,0,0,<p>The purpose of the null is to convert a pro...,140300,None,...,None,None,2019-12-01 03:52:33,None,None,0,0,0,None,None
42230,438740,2,0,61144,2019-12-01 04:26:19,0,0,<p>Related question:</p>\n\n<p>If I have the m...,267322,None,...,None,None,2019-12-01 04:26:19,None,None,0,0,0,None,None
42231,438741,1,0,0,2019-12-01 05:00:39,0,3,<p>I am interested in model selection and wond...,206102,None,...,None,None,2019-12-01 05:00:39,Can I do model selection comparing model compu...,<anova><mixed-model><aic>,0,0,0,None,None
42232,438742,2,0,438698,2019-12-01 05:02:44,0,0,"<p>I would vote for ""deterministic algorithms""...",133755,None,...,None,None,2019-12-01 05:02:44,None,None,0,0,0,None,None


In [8]:
# load the CSV file
users = pd.read_csv('users.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,NaN,NaN,NaN,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,NaN,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,NaN,NaN,NaN,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,NaN,NaN,NaN,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [9]:
# load users into sqlite
users.to_sql('users', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM users'
users = pd.read_sql(sql, con)
users

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18407,157530,101,2017-04-16 22:44:35,Narfanar,2019-11-29 12:50:02,None,None,None,1,17,0,https://i.stack.imgur.com/QtTmm.png?s=128&g=1,1071965
18408,157574,161,2017-04-17 08:53:48,Jan,2019-07-14 08:46:32,None,None,None,15,1,0,https://www.gravatar.com/avatar/3c2b2c7b445c04...,5145151
18409,157575,461,2017-04-17 08:58:30,Jeremy Voisey,2019-11-22 05:29:36,http://www.jvoisey.com,Cyprus,"<p>I work for a small school, where I maintain...",47,10,0,https://i.stack.imgur.com/zCfd5.jpg?s=128&g=1,10485220
18410,157584,1,2017-04-17 10:48:14,Siyabonga Mbonambi,2019-06-03 08:59:00,None,None,None,1,0,0,https://www.gravatar.com/avatar/cd488b3037badd...,10355063


### **TODO**: Drop Duplicates
Look for and drop any duplicates in all 3 of the tables (if they exist). Use Pandas to do this. If you find duplicates, you will need to rewrite the table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. 
    </font></div>

In [10]:
# Check and drop duplicates
comments.duplicated().sum()

0

In [11]:
# Check and drop duplicates
posts.duplicated().sum()

0

In [12]:
# Check and drop duplicates
users.duplicated().sum()

0

## Review the Data

Spend some time reviewing the data.  Understand what data we have, think about how that data can be used to assist in the initiative of understanding how ChatData is used in the real world.  Is the data organised in a way that would lend itself to being manged in a relational database?  How would the different tables be connected?  What are the primary and foreign keys?  Would this give you a 3NF model?

Also think about security and ethics.  Is there personal data in here?  Could individuals be identified through this data?  It it ethical to use the data in this way?  You will be asked to comment on these questions later!

Use the code below to help you.

In [13]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [14]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [15]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [16]:
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


In [17]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [18]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In [19]:
# Check the Null values for comments dataset
comments.isnull().sum().sort_values(ascending = False)/posts.shape[0]*100

Id              0.0
PostId          0.0
Score           0.0
Text            0.0
CreationDate    0.0
UserId          0.0
dtype: float64

In [20]:
# Check the Null values for posts dataset
posts.isnull().sum().sort_values(ascending = False).head(10)/posts.shape[0]*100

LastEditorDisplayName    99.874509
CommunityOwnedDate       99.382014
OwnerDisplayName         98.733248
ClosedDate               95.034806
LastEditDate             60.268978
Tags                     41.845433
Title                    41.845433
Body                      0.016574
FavoriteCount             0.000000
CommentCount              0.000000
dtype: float64

In [21]:
# Check the Null values for users dataset
users.isnull().sum().sort_values(ascending = False).head(7)/users.shape[0]*100

WebsiteUrl         90.924397
AboutMe            78.160982
Location           72.425592
ProfileImageUrl    19.378666
DisplayName         0.010862
Id                  0.000000
Reputation          0.000000
dtype: float64

# Working with Sqlite and SQL Magic
In this section let's spend a little time understanding a bit more about how we can work with Sqlite within Jupyter.

Let's look at 2 ways to query the sqlite database: using SQL Magic or using Pandas.  Either way is fine for this project.

## Writing queries with SQL Magic

You will now need to write some queries to get answers to the questions in the requirements.  

For single-line queries, start the cell with `%sql` and simply enter your query:

In [22]:
# This is an example
%sql SELECT COUNT(*) FROM comments

 * sqlite:///chatdata.db
Done.


COUNT(*)
50000


For multi line sql statements use `%%sql` as follows. This tells Jupyter that *everything* in this cell should be interpreted as sql. So, NO comments other statements are allowed:

In [23]:
%%sql
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text
723182,385124,0,"@BenBolker I don't understand. The fit cannot be done for the negative $y$. So intuitively I'd think that in order to retain the relativity of the data, one'd ideally ""mirror"" or ""flip"" it to the positive axis? Would it be possible to adjust every point individually? Basically e.g. take $abs(y_i)$ instead of minimums."
723183,385124,3,"You can't add *less* than (`-min(y)`), but you could add *more*. I'm going to stop answering now sorry, because **judging what the 'best' approach is depends on much more context that we don't have** (the goal of the analysis, why you need to fit an exponential, your level of computational and statistical sophistication and that of your audience, etc. ...)"
723186,385137,0,"nice. If you felt like doing the work it would be nice to generate an image/contour plot of log-likelihood as a function of (mu, theta) and show the lines corresponding to the two `size` values ."
723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,size=size,log=TRUE)), xlim=c(2,5),ylim=c(0.2,0.5), varnames=c(""mu"",""size""), sys3d=""contour"")abline(h=3.2/7,col=""red"")abline(h=fit.what$theta,col=""blue"")` ... although doesn't look this is actually the answer - mu-hat is independent of theta-hat ... ?"
723188,385134,0,"Don't you mean ""so variance should be $\sigma^2/(n\mu^2)$"""


## Writing queries with Pandas
Another way to write queries is to use pandas:

In [24]:
sql = """
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5
        """
result = pd.read_sql(sql, con)
result

,Id,PostId,Score,Text
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you..."
2,723186,385137,0,nice. If you felt like doing the work it would...
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si..."
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^..."


# Creating Tables with Referential Integrity

When we loaded the csv files into Sqlite database tables, Sqlite created the tables for us behind the scenes. Let's inspect this a bit more.

We can see how Sqlite created the tables by querying the `sqlite_master` table, which Sqlite uses to keep track of what objects have been created in the database:

In [25]:
%%sql
select sql from sqlite_master

 * sqlite:///chatdata.db
Done.


sql
"CREATE TABLE ""comments"" (""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER)"
"CREATE TABLE ""posts"" (""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT)"
"CREATE TABLE ""users"" (""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER)"


The above results show the `CREATE TABLE` statements that could be used by Sqlite to recreate the tables with the exact same structure.

The problem with the CREATE TABLE statements above is that they don't enforce **referential integrity**.  In other words, they don't ensure that every UserId and PostId in the comments table refers to an actual UserId and PostId in the users and posts tables.  At the moment. we can insert any old number here, and even have multiple users with the same Id!  One of the advantages of working with relational databases is that they can enforce the correct uniqueness and relationships in the data, but at the moment we are not using that feature.  So let's fix that...

First, let's drop the original tables:


In [26]:
%%sql
DROP TABLE comments;
DROP TABLE users;
DROP TABLE posts;

 * sqlite:///chatdata.db
Done.
Done.
Done.


[]

Prove that this worked by selecting the names of the tables back.  We should have no tables:

In [27]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name


In Sqlite we need to enable the enforcement of foreign key constraints:

In [28]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

Now recreate the users table with a **primary key constraint** by copying the CREATE TABLE statement from above and adding the NOT NULL PRIMARY KEY clause to the Id:

In [29]:
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
Done.


[]

Now do the same for the posts table:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [30]:
%%sql
CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);

 * sqlite:///chatdata.db
Done.


[]

Now for the comments table.  We need to add the primary key constraint on the id here as we did for users and posts, but we also need to add FOREIGN KEY constraints on the UserId and PostId. Read the documentation here and find our how to do that: https://www.sqlite.org/foreignkeys.html.  Then create the comments table with the correct constraints:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [31]:
%%sql
CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER,
FOREIGN KEY (PostId) REFERENCES posts(Id),
FOREIGN KEY (UserId) REFERENCES users(Id)
);

 * sqlite:///chatdata.db
Done.


[]

Now we can re-insert the data into these constrained tables.  First users:

In [32]:
# Insert data into the new users table
users.to_sql('users', con, if_exists='append', index=False)

18412

Now posts:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [33]:
# Insert data into the new posts table
posts.to_sql('posts', con, if_exists='append', index=False)

42234

Finally comments, which references the users and posts tables:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [34]:
# Insert data into the new comments table
comments.to_sql('comments', con, if_exists='append', index=False)

50000

Now check that we have the 3 new table definitions in Sqlite:

In [35]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name
comments
posts
users


We now have all the data in tables in Sqlite and the tables will enforce the referential integrity.

# Example Query and Pattern for Tasks 2 and 3

As you work through the next tasks, you will need to:

1. Prepare the Sqlite query to answer the question
2. Test it
3. Insert it into the `queries` table, so we have a record of it for others.

This is the process that we want you to follow for this project while completing it. 

Let's see an example of this by answering the following question:

**Which 5 users have viewed the most times and what is the sum of those views per user?**

## Prepare the Sqlite query

First, let's write the query:

In [36]:
sql = """
SELECT Id, SUM(Views) AS TotalViews
    FROM Users
        GROUP BY Id
            ORDER BY TotalViews DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
result

,Id,TotalViews
0,919,85180
1,4253,35119
2,805,34637
3,7290,32639
4,3277,29255


## Test the query
You can optionally prove the query worked by performing the same query in Pandas:

In [37]:
results = users.groupby(['Id']).sum().sort_values('Views', ascending = False)[:5]
results['Views']

Id
919     85180
4253    35119
805     34637
7290    32639
3277    29255
Name: Views, dtype: int64

## Insert the query into the `queries` table
Now we need to put this query into the `queries` table in sqlite. Remember we want these queries to be accessible to everybody that should have access to them. We do not want people writing and rewriting the same queires over and over again. The easiest thing to do is create a dictionary with the values and insert these into the queries table.  Note that the values are provided as lists as we are inserting a list of values (i.e. a number of rows) into the table.  In this case the number of rows is 1, so we have lists of 1 item.

So here, we have a column called 'task' with a list of values, a column called 'action' with a list of values, etc.

In [38]:
query_dict = {
              'task': ['Single Table Queries'],
              'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
              'query': [sql]
             }
query_dict

{'task': ['Single Table Queries'],
 'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
 'query': ['\nSELECT Id, SUM(Views) AS TotalViews\n    FROM Users\n        GROUP BY Id\n            ORDER BY TotalViews DESC\n                LIMIT 5\n                ']}

Now that you have the data structure (query_dict) containing the data, create a pandas dataframe that holds those values:

In [39]:
queries = pd.DataFrame(query_dict)
queries

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


Now load that pandas dataframe (queries) into the sqlite table called queries. In this case, you use append NOT replace. You will be adding to this tables as you go thru this project.

In [40]:
# load query into sqlite
queries.to_sql('queries', con, if_exists='append', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)
queries.head()

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


It is likely that as you iterate thru this notebook you will create some duplicate entries in the query table. Not a big deal. Just tell sql or pandas (much easier in pandas) to drop the duplicates! If you drop the duplicates in Pandas you will have to write the entire dataframe back to sqlite. Otherwise it is changed in memory in Pandas but NOT in sqlite on the disk drive (or SDD). Just do this drop at the end of this notebook so that you are not constantly dealing with this. 

So, to summarise, as you go through the following tasks you need to:

- answer the question in sql
- prove it in pandas (if you want to)
- put the query into the queries table

# Task 1 (continued): Insert the CREATE TABLE Statements into the `queries` Table

Now that we understand how to populate the `queries` table, let's insert the CREATE TABLE statements into it.  First let's define a function to help us insert into the queries table:


In [41]:
# Define a function that will insert into the queries table
def store_query(task, action, query):
    query_dict = {
                  'task': [task],
                  'action': [action],
                  'query': [query]
                 }

    # put query into the query_dict
    queries = pd.DataFrame(query_dict)

    # load query into sqlite
    queries.to_sql('queries', con, if_exists='append', index=False)    

Now we can specify the queries and call the above function to store them.  The first one is done for you.

In [42]:
sql = """
    CREATE TABLE "comments" (
    "Id" INTEGER,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER
    )
    """
store_query("Task 1", "Create table comments", sql)

Let's prove it works by selecting back from the queries table:

In [43]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"


## Insert the other CREATE TABLE statements into the `queries` table.
Follow the above pattern to complete these code cells:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [44]:
# Insert the CREATE TABLE for posts into the queries table
sql = """
    CREATE TABLE "posts" (
    "Id" INTEGER,
    "PostTypeId" INTEGER,
    "AcceptedAnswerId" INTEGER,
    "ParentId" INTEGER,
    "CreationDate" TEXT,
    "Score" INTEGER,
    "ViewCount" INTEGER,
    "Body" TEXT,
    "OwnerUserId" INTEGER,
    "OwnerDisplayName" TEXT,
    "LastEditorUserId" INTEGER,
    "LastEditorDisplayName" TEXT,
    "LastEditDate" TEXT,
    "LastActivityDate" TEXT,
    "Title" TEXT,
    "Tags" TEXT,
    "AnswerCount" INTEGER,
    "CommentCount" INTEGER,
    "FavoriteCount" INTEGER,
    "ClosedDate" TEXT,
    "CommunityOwnedDate" TEXT
    )
    """
store_query("Task 1", "Create table posts", sql)

In [45]:
# Insert the CREATE TABLE for users into the queries table
sql = """
    CREATE TABLE "users" (
    "Id" INTEGER,
    "Reputation" INTEGER,
    "CreationDate" TEXT,
    "DisplayName" TEXT,
    "LastAccessDate" TEXT,
    "WebsiteUrl" TEXT,
    "Location" TEXT,
    "AboutMe" TEXT,
    "Views" INTEGER,
    "UpVotes" INTEGER,
    "DownVotes" INTEGER,
    "ProfileImageUrl" TEXT,
    "AccountId" INTEGER
    )
    """
store_query("Task 1", "Create table users", sql)

## Count the Number of Rows in Each Table
Run some queries to count the number of rows in each of the tables.  Don't forget to insert the query into the queries table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [46]:
# Count the number of rows in the comments table
sql = """
SELECT COUNT(*)
    FROM comments
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Count number of rows in table comments", sql)
result

,COUNT(*)
0,50000


In [47]:
# Count the number of rows in the users table
sql = """
SELECT COUNT(*)
    FROM users
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Count number of rows in table users", sql)
result

,COUNT(*)
0,18412


In [48]:
# Count the number of rows in the posts table
sql = """
SELECT COUNT(*)
    FROM posts
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Count number of rows in posts users", sql)
result

,COUNT(*)
0,42234


## Do some Random Checks on the Data
Let's write some queries that select 5 random rows from each table.  The queries are provided here:

    select * from Comments order by random() limit 5;
    select * from Users order by random() limit 5;
    select * from Posts order by random() limit 5;

Enter the queries into the code cells below and insert the queries into the `queries` table in the same was as you did for the CREATE TABLE statements.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [49]:
# Run the query to select 5 random rows from the posts table
sql = """
SELECT *
    FROM posts
        ORDER BY random()
            LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Random checks on posts", sql)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,414634,1,0,0,2019-06-25 11:29:30,1,20,"<p>I'm working through <a href=""https://pagepi...",251893,None,...,None,2019-06-25 13:55:12,2019-06-25 13:58:40,Models converging with different versions of lme4,<r><lme4-nlme>,1,0,0,None,None
1,394796,1,396286,0,2019-02-28 00:27:24,1,754,<p>Should mytime series be stationary to use A...,74737,None,...,None,2019-02-28 00:58:35,2019-03-08 01:51:38,Should my time series be stationary to use ARI...,<time-series><arima>,5,0,1,None,None
2,418584,1,419203,0,2019-07-22 10:27:41,15,1677,<p>I am trying to create a figure which shows ...,254367,None,...,None,2019-09-12 13:12:31,2019-09-12 13:12:31,What kind of curve (or model) should I fit to ...,<regression><modeling><curve-fitting><percentage>,7,9,6,None,None
3,428570,2,0,285023,2019-09-24 23:53:42,1,0,<p>I wrote a little Python helper to help with...,102505,None,...,None,None,2019-09-24 23:53:42,None,None,0,0,0,None,None
4,432087,2,0,32272,2019-10-18 16:03:26,0,0,"<p>If <span class=""math-container"">$V &gt; U &...",263068,None,...,None,None,2019-10-18 16:03:26,None,None,0,0,0,None,None


In [50]:
# Run the query to select 5 random rows from the posts comments
sql = """
SELECT *
    FROM comments
        ORDER BY random()
            LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Random checks on comments", sql)
result

,Id,PostId,Score,Text,CreationDate,UserId
0,762434,408027,0,interesting. Is it possible to deduce nonstati...,2019-05-13 10:53:55,20980
1,780635,28528,0,Maybe treat as a point process rather than a t...,2019-07-20 22:04:43,11887
2,730192,388790,0,"That makes no sense, either mathematically or ...",2019-01-23 19:01:52,919
3,781867,419047,0,@Glen_b so you’re saying that I should model t...,2019-07-25 06:59:32,254670
4,734943,391422,0,The 0 data are real values which were measured...,2019-02-08 08:14:46,235605


In [51]:
# Run the query to select 5 random rows from the users table
sql = """
SELECT *
    FROM users
        ORDER BY random()
            LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 1", "Random checks on users", sql)
result

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,249649,1,2019-06-01 08:32:05,FeynmanBestMan,2019-07-22 09:52:03,None,None,None,3,0,0,https://www.gravatar.com/avatar/ef5622fd7c24e2...,13886764
1,253725,1,2019-07-15 18:09:32,user253725,2019-07-16 06:57:26,None,None,None,0,0,0,None,16322628
2,9594,121,2012-03-04 17:01:27,Seb Destercke,2019-11-12 07:44:11,https://www.hds.utc.fr/~sdesterc/dokuwiki/start,"Compiègne, France",None,1,1,0,None,549938
3,263063,31,2019-10-18 14:47:34,Ollennjj,2019-10-20 03:22:34,None,None,None,2,0,0,https://lh6.googleusercontent.com/-69W1CU41kIg...,11965215
4,192441,125,2018-01-22 11:40:57,Ronald,2019-11-14 04:02:54,None,India,<p>Hello i am new here.</p>\n,11,12,0,https://graph.facebook.com/1888177307918916/pi...,12630122


In [52]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Create table posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in posts users,SELECT COUNT(*) FROM posts
Task 1,Random checks on posts,SELECT * FROM posts ORDER BY random() LIMIT 5
Task 1,Random checks on comments,SELECT * FROM comments ORDER BY random() LIMIT 5
Task 1,Random checks on users,SELECT * FROM users ORDER BY random() LIMIT 5


# Task 2: Create Single Table Queries


## Lifecycle Stage: Analyze
We can now start the analysis with our single-table queries.  First we need to create a new computed column to help with one of the queries.  The code below creates a column called LEN_BODY which is the length of the BODY text:


In [53]:
%%sql
ALTER TABLE POSTS ADD COLUMN LEN_BODY INT

 * sqlite:///chatdata.db
Done.


[]

In [54]:
%%sql
UPDATE POSTS SET LEN_BODY = LENGTH(BODY)

 * sqlite:///chatdata.db


42234 rows affected.


[]

### Single Table Queries
From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>

In [55]:
# How many posts have 0 comments?
sql = """
SELECT COUNT(*)
    FROM posts
        WHERE CommentCount = 0
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many posts have 0 comments?", sql)
result

,COUNT(*)
0,21713


In [56]:
# How many posts have 1 comments?
sql = """
SELECT COUNT(*)
    FROM posts
        WHERE CommentCount = 1
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many posts have 1 comments?", sql)
result

,COUNT(*)
0,6460


In [57]:
# How many posts have 2 or more comments?
sql = """
SELECT COUNT(*)
    FROM posts
        WHERE CommentCount >= 2
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many posts have 2 comments?", sql)
result

,COUNT(*)
0,14061


In [58]:
# Find the 5 posts with the highest viewcount
sql = """
SELECT *
    FROM posts
        ORDER BY ViewCount DESC
            LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "Find the 5 posts with the highest viewcount", sql)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,388566,1,388582,0,2019-01-22 15:16:47,56,19542,"<ul>\n<li>Statement One (S1): ""One in 80 death...",228214,None,...,2019-01-22 21:40:39,2019-01-24 17:09:47,"Is it wrong to rephrase ""1 in 80 deaths is cau...",<interpretation><risk>,9,15,15,None,None,2270
1,394118,1,394128,0,2019-02-24 14:07:11,64,16317,<p>A human child at age 2 needs around 5 insta...,107213,None,...,2019-02-25 22:40:22,2019-03-03 17:37:05,Why do neural networks need so many training e...,<neural-networks><neuroscience>,12,24,38,None,None,512
2,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811
3,398646,1,398653,0,2019-03-21 01:19:52,61,9850,<p>The title of the Comment in Nature <a href=...,163067,None,...,2019-03-22 22:14:04,2019-03-30 19:35:27,"What does ""Scientists rise up against statisti...",<statistical-significance><p-value><bias>,10,7,34,None,None,2148
4,434128,1,434579,0,2019-11-01 13:07:36,73,6718,<p>I am designing a one year program in data a...,14188,None,...,None,2019-11-26 00:59:15,Famous statistical wins and horror stories for...,<mathematical-statistics><data-visualization><...,13,7,70,None,2019-11-01 15:12:41,1172


In [59]:
# Find the top 5 posts with the highest scores
sql = """
SELECT *
    FROM posts
        ORDER BY Score DESC
            LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "Find the top 5 posts with the highest scores", sql)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,431397,2,0,431370,2019-10-14 14:29:36,101,0,<p>Yes there is. Generally it is termed <stron...,142976,None,...,None,2019-10-14 14:29:36,None,None,0,0,0,None,None,269
1,394128,2,0,394118,2019-02-24 15:44:44,100,0,<p>I caution against expecting strong resembla...,22311,None,...,2019-03-03 17:37:05,2019-03-03 17:37:05,None,None,0,15,0,None,None,5829
2,426878,2,0,426873,2019-09-11 23:23:31,93,0,<p><strong>tl;dr</strong> Even though this is ...,119015,None,...,2019-09-13 14:02:28,2019-09-13 14:02:28,None,None,0,6,0,None,None,4627
3,388578,2,0,388566,2019-01-22 15:48:47,80,0,"<p>To me ""1 in 80 deaths..."" is by far the cle...",227039,None,...,None,2019-01-22 15:48:47,None,None,0,11,0,None,None,572
4,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811


In [60]:
# What are the 5 most frequent scores on posts?
sql = """
SELECT Score, COUNT(*) AS Frequency
    FROM posts
        GROUP BY Score
            ORDER BY Frequency DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What are the 5 most frequent scores on posts?", sql)
result

,Score,Frequency
0,0,19888
1,1,11867
2,2,5094
3,3,2228
4,4,1059


In [61]:
# How many posts have the keyword "data" in their tags?
sql = """
SELECT COUNT(*)
    FROM posts
        WHERE Tags LIKE '%data%'
                """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many posts have the keyword 'data' in their tags?", sql)
result

,COUNT(*)
0,2242


In [62]:
# What are the 5 most frequent commentcount for posts?
sql = """
SELECT CommentCount, COUNT(*) AS Frequency
    FROM posts
        GROUP BY CommentCount
            ORDER BY Frequency DESC
                LIMIT 5
                    """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What are the 5 most frequent commentcount for posts?", sql)
result

,CommentCount,Frequency
0,0,21713
1,1,6460
2,2,4966
3,3,3063
4,4,2026


In [63]:
# How many posts have an accepted answer?
sql = """
SELECT COUNT(*)
    FROM posts
        WHERE AcceptedAnswerId != 0
            """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many posts have an accepted answer?", sql)
result

,COUNT(*)
0,5341


In [64]:
# What is the average reputation of table users?
sql = """
SELECT ROUND(AVG(Reputation),2) AS Average_Reputation
    FROM users
            """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What is the average reputation of table users?", sql)
result

,Average_Reputation
0,312.35


In [65]:
# What are the min and max reputation of users?
sql = """
SELECT MIN(Reputation) AS Min_Reputation, MAX(Reputation) AS Max_Reputation
    FROM users
            """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What are the min and max reputation of users?", sql)
result

,Min_Reputation,Max_Reputation
0,1,228662


In [66]:
# What is the length of the body of 5 most viewed posts?
sql = """
SELECT Id, Title, ViewCount, LEN_BODY AS Body_Length
    FROM posts
        ORDER BY ViewCount DESC
            LIMIT 5
            """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What is the length of the body of 5 most viewed posts?", sql)
result

,Id,Title,ViewCount,Body_Length
0,388566,"Is it wrong to rephrase ""1 in 80 deaths is cau...",19542,2270
1,394118,Why do neural networks need so many training e...,16317,512
2,431370,Is there a name for the phenomenon of false po...,11723,811
3,398646,"What does ""Scientists rise up against statisti...",9850,2148
4,434128,Famous statistical wins and horror stories for...,6718,1172


In [67]:
# How many different locations are there in the users table?
sql = """
SELECT COUNT(DISTINCT Location) AS Number_of_Unique_Locations
    FROM users
            """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "How many different locations are there in the users table?", sql)
result

,Number_of_Unique_Locations
0,1900


In [68]:
# What are the top 5 locations of users?
sql = """
SELECT Location, COUNT(*) AS Frequency
    FROM users
        WHERE Location IS NOT NULL
            GROUP BY Location
                ORDER BY Frequency DESC
                    LIMIT 5
                    """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "What are the top 5 locations of users?", sql)
result

,Location,Frequency
0,Germany,117
1,India,100
2,United States,69
3,"Paris, France",66
4,"London, United Kingdom",63


In [69]:
# Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.
sql = """
SELECT 
    CASE CAST(strftime('%w', CreationDate) AS INTEGER)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
    END AS Day_of_Week,
    ROUND(SUM(ViewCount) * 100.0 / (SELECT SUM(ViewCount) FROM posts),2) AS Percentage_ViewCount
        FROM posts
            GROUP BY CAST(strftime('%w', CreationDate) AS INTEGER)
                ORDER BY Percentage_ViewCount DESC;
                    """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 2", "Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.", sql)
result

,Day_of_Week,Percentage_ViewCount
0,Thursday,16.82
1,Wednesday,16.77
2,Tuesday,16.26
3,Monday,15.81
4,Friday,13.56
5,Sunday,11.90
6,Saturday,8.89


# Task 3: Cross Table Queries

## Lifecycle Stage: Analyze
Let's continue the analysis with our multi-table queries.  

### Cross Table Queries

From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>



In [70]:
# How many posts have been created by a user that has a filled out the "AboutMe" section?
sql = """
SELECT COUNT(p.Id) AS Num_of_Posts
    FROM posts p
        LEFT JOIN users u ON p.OwnerUserId = u.Id
            WHERE u.AboutMe IS NOT NULL
                    """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "How many posts have been created by a user that has a filled out the 'AboutMe' section?", sql)
result

,Num_of_Posts
0,17189


In [71]:
# Considering only the users with an "AboutMe," how many posts are there per user?
sql = """
SELECT ROUND(CAST(COUNT(p.Id) AS FLOAT) / (SELECT COUNT(DISTINCT Id) FROM users WHERE AboutMe IS NOT NULL), 2) AS Average_posts_per_user
    FROM posts p
        LEFT JOIN users u ON p.OwnerUserId = u.Id
            WHERE u.AboutMe IS NOT NULL;
                    """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "Considering only the users with an 'AboutMe,' how many posts are there per user?", sql)
result

,Average_posts_per_user
0,4.27


In [72]:
# Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?
sql = """
SELECT p.Id AS PostId, COUNT(c.Id) AS Num_of_comments
    FROM posts p
        LEFT JOIN comments c ON p.Id = c.PostId
            GROUP BY p.Id
                ORDER BY Num_of_comments DESC
                    LIMIT 10;
                        """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?", sql)
result

,PostId,Num_of_comments
0,386853,66
1,386556,34
2,395232,31
3,418910,31
4,402987,27
5,386075,26
6,394118,24
7,398828,23
8,402950,23
9,396111,22


In [73]:
# What are the Top 10 posts which have the highest cummulative (post score + comment score) score? 
sql = """
SELECT p.id AS PostId, p.Score AS Post_score, SUM(c.Score) AS Sum_comments_score, SUM(c.Score) + p.Score AS Cummulative_score
    FROM posts p
        LEFT JOIN comments c ON p.Id = c.PostId
            GROUP BY p.Id
                ORDER BY Cummulative_score DESC
                    LIMIT 10;
                        """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "What are the Top 10 posts which have the highest cummulative (post score + comment score) score?", sql)
result

,PostId,Post_score,Sum_comments_score,Cummulative_score
0,394118,64,242,306
1,394128,100,69,169
2,388578,80,61,141
3,398653,65,46,111
4,388566,56,45,101
5,398646,61,38,99
6,421677,37,59,96
7,420526,45,47,92
8,400317,43,29,72
9,388582,67,2,69


In [74]:
# Who are the top 10 users who comment the most? 
sql = """
SELECT u.Id AS User_Id, COUNT(c.Id) AS Num_of_comments, u.Reputation AS User_reputation
    FROM comments c
        LEFT JOIN users u ON u.Id = c.UserId
            GROUP BY u.Id, u.Reputation
                ORDER BY User_reputation DESC
                    LIMIT 10;
                        """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "Who are the top 10 users who comment the most?", sql)
result

,User_Id,Num_of_comments,User_reputation
0,805,1153,228662
1,919,3301,223056
2,7290,370,115531
3,686,349,85077
4,28666,158,75024
5,35989,470,71548
6,7224,424,65999
7,4253,255,59952
8,1352,422,59160
9,22311,467,51155


In [75]:
# Who are the top 10 users who post the most? 
sql = """
SELECT u.Id AS User_Id, COUNT(p.Id) AS Num_of_posts, u.Reputation AS User_reputation
    FROM posts p
        LEFT JOIN users u ON u.Id = p.OwnerUserId
            GROUP BY u.Id, u.Reputation
                ORDER BY User_reputation DESC
                    LIMIT 10;
                        """
result = pd.read_sql(sql, con) # con is the connection to the database
store_query("Task 3", "Who are the top 10 users who post the most?", sql)
result

,User_Id,Num_of_posts,User_reputation
0,805,230,228662
1,919,203,223056
2,7290,35,115531
3,686,386,85077
4,28666,8,75024
5,35989,230,71548
6,7224,233,65999
7,4253,71,59952
8,1352,285,59160
9,22311,140,51155


# Task 4: Check the Queries Table

Now let's tidy up and check the queries table.

First let's check it's contents:

In [76]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 1,Create table posts,"CREATE TABLE ""posts"" ( ""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT )"
Task 1,Create table users,"CREATE TABLE ""users"" ( ""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER )"
Task 1,Count number of rows in table comments,SELECT COUNT(*) FROM comments
Task 1,Count number of rows in table users,SELECT COUNT(*) FROM users
Task 1,Count number of rows in posts users,SELECT COUNT(*) FROM posts
Task 1,Random checks on posts,SELECT * FROM posts ORDER BY random() LIMIT 5
Task 1,Random checks on comments,SELECT * FROM comments ORDER BY random() LIMIT 5
Task 1,Random checks on users,SELECT * FROM users ORDER BY random() LIMIT 5


## Drop Duplicates

You likely have some duplicates. Lets drop them. 

In [77]:
# Read the queries table into pandas
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)

# Drop duplicates
queries.drop_duplicates(inplace = True) # drop duplicates

## Case Issues

Remember, SQL is case insensitive. Pandas IS case sensitive. Lets deal with this now by making all of the text uppercase. 

In [78]:
for col in queries.columns:
    queries[col] = queries[col].str.upper()
    
queries

,task,action,query
0,SINGLE TABLE QUERIES,WHICH 5 USERS HAVE VIEWED THE MOST TIMES AND W...,"\nSELECT ID, SUM(VIEWS) AS TOTALVIEWS\n FRO..."
1,TASK 1,CREATE TABLE COMMENTS,"\n CREATE TABLE ""COMMENTS"" (\n ""ID"" INTE..."
2,TASK 1,CREATE TABLE POSTS,"\n CREATE TABLE ""POSTS"" (\n ""ID"" INTEGER..."
3,TASK 1,CREATE TABLE USERS,"\n CREATE TABLE ""USERS"" (\n ""ID"" INTEGER..."
4,TASK 1,COUNT NUMBER OF ROWS IN TABLE COMMENTS,\nSELECT COUNT(*)\n FROM COMMENTS\n ...
5,TASK 1,COUNT NUMBER OF ROWS IN TABLE USERS,\nSELECT COUNT(*)\n FROM USERS\n ...
6,TASK 1,COUNT NUMBER OF ROWS IN POSTS USERS,\nSELECT COUNT(*)\n FROM POSTS\n ...
7,TASK 1,RANDOM CHECKS ON POSTS,\nSELECT *\n FROM POSTS\n ORDER BY R...
8,TASK 1,RANDOM CHECKS ON COMMENTS,\nSELECT *\n FROM COMMENTS\n ORDER B...
9,TASK 1,RANDOM CHECKS ON USERS,\nSELECT *\n FROM USERS\n ORDER BY R...


In [79]:
# Write the now deduped uppercase dataframe back to sqlite and replace the table
queries.to_sql('queries', con, if_exists='replace', index=False)

31

## Use Case

Now that we have this queries table, lets give you some ideas about how you would use it.

Suppose you wanted to find all of the queries where you did a GROUP BY:

In [80]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%GROUP BY%'

 * sqlite:///chatdata.db
Done.


query
"SELECT ID, SUM(VIEWS) AS TOTALVIEWS FROM USERS GROUP BY ID ORDER BY TOTALVIEWS DESC LIMIT 5"
"SELECT SCORE, COUNT(*) AS FREQUENCY FROM POSTS GROUP BY SCORE ORDER BY FREQUENCY DESC LIMIT 5"
"SELECT COMMENTCOUNT, COUNT(*) AS FREQUENCY FROM POSTS GROUP BY COMMENTCOUNT ORDER BY FREQUENCY DESC LIMIT 5"
"SELECT LOCATION, COUNT(*) AS FREQUENCY FROM USERS WHERE LOCATION IS NOT NULL GROUP BY LOCATION ORDER BY FREQUENCY DESC LIMIT 5"
"SELECT CASE CAST(STRFTIME('%W', CREATIONDATE) AS INTEGER) WHEN 0 THEN 'SUNDAY' WHEN 1 THEN 'MONDAY' WHEN 2 THEN 'TUESDAY' WHEN 3 THEN 'WEDNESDAY' WHEN 4 THEN 'THURSDAY' WHEN 5 THEN 'FRIDAY' WHEN 6 THEN 'SATURDAY' END AS DAY_OF_WEEK, ROUND(SUM(VIEWCOUNT) * 100.0 / (SELECT SUM(VIEWCOUNT) FROM POSTS),2) AS PERCENTAGE_VIEWCOUNT FROM POSTS GROUP BY CAST(STRFTIME('%W', CREATIONDATE) AS INTEGER) ORDER BY PERCENTAGE_VIEWCOUNT DESC;"
"SELECT P.ID AS POSTID, COUNT(C.ID) AS NUM_OF_COMMENTS FROM POSTS P LEFT JOIN COMMENTS C ON P.ID = C.POSTID GROUP BY P.ID ORDER BY NUM_OF_COMMENTS DESC LIMIT 10;"
"SELECT P.ID AS POSTID, P.SCORE AS POST_SCORE, SUM(C.SCORE) AS SUM_COMMENTS_SCORE, SUM(C.SCORE) + P.SCORE AS CUMMULATIVE_SCORE FROM POSTS P LEFT JOIN COMMENTS C ON P.ID = C.POSTID GROUP BY P.ID ORDER BY CUMMULATIVE_SCORE DESC LIMIT 10;"
"SELECT U.ID AS USER_ID, COUNT(C.ID) AS NUM_OF_COMMENTS, U.REPUTATION AS USER_REPUTATION FROM COMMENTS C LEFT JOIN USERS U ON U.ID = C.USERID GROUP BY U.ID, U.REPUTATION ORDER BY USER_REPUTATION DESC LIMIT 10;"
"SELECT U.ID AS USER_ID, COUNT(P.ID) AS NUM_OF_POSTS, U.REPUTATION AS USER_REPUTATION FROM POSTS P LEFT JOIN USERS U ON U.ID = P.OWNERUSERID GROUP BY U.ID, U.REPUTATION ORDER BY USER_REPUTATION DESC LIMIT 10;"


## Now Your Turn
Find the queries that have 'DISTINCT' in them. You can do it with the %sql command or with Pandas and sql.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [81]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%DISTINCT%'

 * sqlite:///chatdata.db
Done.


query
SELECT COUNT(DISTINCT LOCATION) AS NUMBER_OF_UNIQUE_LOCATIONS FROM USERS
"SELECT ROUND(CAST(COUNT(P.ID) AS FLOAT) / (SELECT COUNT(DISTINCT ID) FROM USERS WHERE ABOUTME IS NOT NULL), 2) AS AVERAGE_POSTS_PER_USER FROM POSTS P LEFT JOIN USERS U ON P.OWNERUSERID = U.ID WHERE U.ABOUTME IS NOT NULL;"


# Close SQLite

It is good practise to close all relational databases as soon as you are finished updating them.

In [82]:
con.close()

# All Done!

Great job. You now have a good idea for how to use sql and pandas with sql. You can create your own databases from csv files and you can do extensive querying using sql. These are valuable skills that will take you a long ways in todays technological world.